#start


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
from multiprocessing.dummy import Pool as ThreadPool
import tqdm

In [3]:

# Define paths
csv_image_mapping = [
    {
        'csv_path': '/content/drive/MyDrive/Tomato_dataset/phone2(batch3)/annotations/phone2(batch3).csv',
        'image_dir': '/content/drive/MyDrive/Tomato_dataset/phone2(batch3)'
    },
    {
        'csv_path': '/content/drive/MyDrive/Tomato_dataset/phone2(batch4)/annotations/phone2(batch4).csv',
        'image_dir': '/content/drive/MyDrive/Tomato_dataset/phone2(batch4)'
    },
    {
        'csv_path': '/content/drive/MyDrive/Tomato_dataset/phone2(batch5)/annotations/phone2(batch5).csv',
        'image_dir': '/content/drive/MyDrive/Tomato_dataset/phone2(batch5)'
    },
    {
        'csv_path': '/content/drive/MyDrive/Tomato_dataset/camera_ripe(batch1)/annotations/camera_ripe(batch1).csv',
        'image_dir': '/content/drive/MyDrive/Tomato_dataset/camera_ripe(batch1)'
    }
]
output_dir = '/content/drive/MyDrive/Tomato_dataset/cnn_crops'

In [4]:


output_dir = '/content/drive/MyDrive/Tomato_dataset/cnn_crops'

ripe_labels = ['R.healthy', 'R.lateblight', 'R.spots', 'R.pests', 'R.ber']
green_labels = ['G.healthy', 'G.lateblight', 'G.spots', 'G.pests', 'G.ber']

# Example mapping
# csv_image_mapping = [{'csv_path': 'path/to/csv1.csv', 'image_dir': 'path/to/images1'},
#                      {'csv_path': 'path/to/csv2.csv', 'image_dir': 'path/to/images2'}]

# -----------------------------
# LOAD AND FILTER DATA
# -----------------------------
dfs = [pd.read_csv(m['csv_path']).assign(image_dir=m['image_dir']) for m in csv_image_mapping]
data = pd.concat(dfs, ignore_index=True)[
    ['image_name','label_name','bbox_x','bbox_y','bbox_width','bbox_height','image_dir']
]
data = data[data['label_name'].isin(ripe_labels + green_labels)]

# -----------------------------
# CREATE OUTPUT DIRECTORIES IF MISSING
# -----------------------------
for ripeness in ['ripe', 'green']:
    labels = ripe_labels if ripeness == 'ripe' else green_labels
    for label in labels:
        dir_path = os.path.join(output_dir, ripeness, label.replace('.', '_'))
        os.makedirs(dir_path, exist_ok=True)

# -----------------------------
# TRAIN/VAL SPLIT
# -----------------------------
unique_images = data['image_name'].unique()
train_images, val_images = train_test_split(unique_images, test_size=0.2, random_state=42)
train_set, val_set = set(train_images), set(val_images)

# -----------------------------
# PREPARE ROWS FOR THREADING
# -----------------------------
rows = [(r['image_dir'], r['image_name'], r['label_name'], r['bbox_x'], r['bbox_y'], r['bbox_width'], r['bbox_height'])
        for r in data.to_dict(orient='records')]

# -----------------------------
# PROCESS FUNCTION
# -----------------------------
def process_row(row):
    image_dir, image_name, label_name, x, y, w, h = row
    img_path = os.path.join(image_dir, image_name)
    if not os.path.exists(img_path):
        return
    img = cv2.imread(img_path)
    if img is None:
        return
    x, y, w, h = map(int, (x, y, w, h))
    crop = img[y:y+h, x:x+w]
    if crop.size == 0:
        return
    crop = cv2.resize(crop, (224, 224))

    # Determine ripeness and split
    ripeness = 'ripe' if label_name.startswith('R') else 'green'
    split = 'train' if image_name in train_set else 'val'
    label_dir = label_name.replace('.', '_')

    # Check if split folder exists; create if missing
    final_dir = os.path.join(output_dir, ripeness, split, label_dir)
    os.makedirs(final_dir, exist_ok=True)

    # Save crop
    save_path = os.path.join(final_dir, f"{image_name.replace('.jpg', f'_{x}_{y}.jpg')}")
    cv2.imwrite(save_path, crop)

# -----------------------------
# PARALLEL EXECUTION
# -----------------------------
num_threads = 8  # Adjust depending on your CPU
with ThreadPool(num_threads) as pool:
    list(tqdm.tqdm(pool.imap(process_row, rows), total=len(rows)))

print(f"✅ Cropped datasets saved to {output_dir}")


100%|██████████| 1631/1631 [02:51<00:00,  9.49it/s]

✅ Cropped datasets saved to /content/drive/MyDrive/Tomato_dataset/cnn_crops


In [ ]:
import os
import shutil

# Base dataset path
base_dir = "/content/drive/MyDrive/Tomato_dataset/cnn_crops"
backup_dir = os.path.join(base_dir, "removed_classes")

# Ensure backup folder exists
os.makedirs(backup_dir, exist_ok=True)

# Folders to move
to_move = [
    f"{base_dir}/ripe/train/R_pests",
    f"{base_dir}/ripe/val/R_pests",
    f"{base_dir}/green/train/G_pests",
    f"{base_dir}/green/val/G_pests",
]

# Move images
for folder in to_move:
    if os.path.exists(folder):
        # Get all files in folder
        for filename in os.listdir(folder):
            src_path = os.path.join(folder, filename)
            dst_path = os.path.join(backup_dir, f"{os.path.basename(folder)}_{filename}")
            shutil.move(src_path, dst_path)
        print(f"✅ Moved images from: {folder} → {backup_dir}")
        # Optionally remove the empty folder
        os.rmdir(folder)
    else:
        print(f"⚠️ Folder not found: {folder}")

print("\n✨ Done! R_pests and G_pests images moved to 'removed_classes'.")
